In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from scipy.stats import mode
import sklearn.metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from TumorTruths import TumorTruths

In [2]:
svnCalls = pd.read_csv('SNVCalls_IS1.txt', delimiter='\t', dtype={'CHROM':pd.np.str})
truths = TumorTruths('synthetic.challenge.set1.tumor.all.truth.vcf')
genomicFeatures = pd.read_csv('GenomicFeatures_IS1.txt', delimiter='\t', dtype={'CHROM':pd.np.str})

In [3]:
resultsArray = truths.GetTruths(svnCalls)
print len(svnCalls.columns)

122


In [4]:
mostCommonPredictions = mode(svnCalls[svnCalls.columns[3:-1]], axis=1)[0]
sklearn.metrics.f1_score(mostCommonPredictions, resultsArray)

0.96532793990176247

In [5]:
noSomatic = [i for i in range(len(resultsArray)) if resultsArray[i] == 0]
nonSomaticSvns = svnCalls.iloc[noSomatic]
nonSomaticGenomicFeatures = genomicFeatures[genomicFeatures['END'].isin(nonSomaticSvns['END'])]
numFalsePositives = nonSomaticSvns[nonSomaticSvns.columns[3:-1]].sum(axis=1)

In [6]:
def ValuesMatch(prediction, result):
    return 1 if prediction == result else 0

valuesMatchFunc = np.vectorize(ValuesMatch)  
correctPredictions = valuesMatchFunc(mostCommonPredictions[:,0], resultsArray)

In [102]:
features = nonSomaticGenomicFeatures
# features = genomicFeatures
predictions = numFalsePositives.values
# predictions = correctPredictions
# def Test(features, predictions):
model = LinearRegression()
#     X = (genomicFeatures['GCcontent'] - 0.5).values.reshape((len(genomicFeatures), 1))

X1 = (features['NormalCoverage'] / (features['NormalCoverage'] + features['TumourCoverage'])) - 0.5
X2 = features['GCcontent'] - 0.5
X3 = features['Distance']
X = np.ndarray((len(features), 2))
X[:,0] = X1 * X1
X[:,1] = X2 * X2
model.fit(X, predictions)
print np.min(features['Distance'])
print np.max(features['Distance'])
# print features.columns
# print np.std(features['Distance'])
# print features['Distance']

0
3335821


In [100]:
# model = Test(nonSomaticGenomicFeatures, numFalsePositives)
# print model.coef_
# print correctPredictions.shape
# print np.zeros((100,)).T.shape
# print correctPredictions
print model.coef_
print model.intercept_
print model.score(X, predictions)

[ 14.92168833   6.93074815]
1.5759031614
0.0138441172373


In [12]:
print X.T.shape
print predictions.values.shape

(1, 211001)
(211001,)


In [ ]:
lowGcContent = genomicFeatures[genomicFeatures['GCcontent'] < 0.14]['END']
# print svnCalls[svnCalls['END'].isin(lowGcContent)]
for caller in lowGcContent:
    svnCalls[caller] = np.zeros(len(svnCalls))

In [ ]:
# mostCommonPredictions = mode(svnCalls[svnCalls.columns[3:-1]], axis=1)[0]
# sklearn.metrics.f1_score(mostCommonPredictions, resultsArray)
print mode(svnCalls[svnCalls.columns[3:-1]])

In [ ]:
col =  genomicFeatures['TumourCoverage'] / (genomicFeatures['NormalCoverage'] + genomicFeatures['TumourCoverage'])
len(genomicFeatures[col < 0.13])